# Clasificación MNIST con red convolucional

MNIST es un ejemplo clásico de reconocimiento de dígitos escritos a mano. Se utiliza la base de datos MNIST que contiene 60,000 imágenes de entrenamiento y 10,000 imágenes de prueba. Cada imagen es de 28x28 pixeles y cada pixel tiene un valor entre 0 y 255.

Este dataset marcó un hito en la historia de la IA, con el que [en 1998 el equipo de Yann LeCun utilizó una red neuronal convolucional para conseguir un error de 0.8% en el reconocimiento de dígitos](https://www.youtube.com/watch?v=H0oEr40YhrQ), usando la arquitectura LeNet-5.

Es el mismo ejemplo con el que se explica la [teoría sobre redes neuronales en el video de 3Brown1Blue](https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi) ([versión doblada a español](https://www.youtube.com/watch?v=jKCQsndqEGQ)).

In [6]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision import datasets, transforms
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm
from sklearn.metrics import accuracy_score

## Carga del dataset

A menudo usaremos más de una transformación para preprocesar los datos. Por ejemplo, en el caso de las imágenes, a menudo se normalizan y se redimensionan. Para hacer esto de manera eficiente, podemos usar la clase `Compose` de `torchvision.transforms`. 

In [7]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Transformaciones para convertir a tensor
transform = transforms.Compose([transforms.ToTensor()])

# Cargamos el dataset MNIST
train_data_full = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST('./data', train=False, download=True, transform=transform)

# Número total de muestras de entrenamiento
num_train = len(train_data_full)
val_ratio = 0.2  # Por ejemplo, 20% para validación

# Tamaños de split
num_val = int(num_train * val_ratio)
num_train = num_train - num_val

# División de datos en entrenamiento y validación
train_data, val_data = random_split(train_data_full, [num_train, num_val])

# Dataloaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Número de clases
num_classes = len(train_data_full.classes)  # o también: len(train_data_full.classes)
print(f"Número de clases en MNIST: {num_classes}")

#quiero dividir x e y de val_data
def split_data(data):
    x = torch.stack([item[0] for item in data])
    y = torch.tensor([item[1] for item in data])
    return x, y




Número de clases en MNIST: 10


## Definición del modelo

In [8]:
from torch.nn import functional as F

class CNN(nn.Module):
    def __init__(self, conv_filters, hidden_layers, hidden_units, output_size=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, conv_filters, kernel_size=3, padding=1)   # (28x28) -> (28x28)
        self.conv2 = nn.Conv2d(conv_filters, conv_filters * 2, kernel_size=3, padding=1)  # (14x14) -> (14x14)
        self.pool = nn.MaxPool2d(2, 2)  # reduce a la mitad cada vez

        self.flattened_size = (conv_filters * 2) * 7 * 7  # después de 2 poolings

        fc_layers = []
        in_features = self.flattened_size
        for _ in range(hidden_layers):
            fc_layers.append(nn.Linear(in_features, hidden_units))
            fc_layers.append(nn.ReLU())
            in_features = hidden_units
        fc_layers.append(nn.Linear(in_features, output_size))
        self.fc_net = nn.Sequential(*fc_layers)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 28x28 → 14x14
        x = self.pool(F.relu(self.conv2(x)))  # 14x14 → 7x7
        x = x.view(x.size(0), -1)
        x = self.fc_net(x)
        return x

  

## Entrenamiento del modelo

### Definición de la función de pérdida y el optimizador

Definimos la función de perdida y el optimizador. En este caso usaremos el optimizador `optim.Adam`. Adam es una variante del descenso de gradiente estocástico que calcula tasas de aprendizaje individuales para diferentes parámetros.

In [9]:
# Grid de hiperparámetros
param_grid = {
    'conv_filters': [16, 32],
    'hidden_layers': [1, 2],
    'hidden_units': [64, 128],
    'lr': [0.001],
    'batch_size': [64]
}
grid = list(ParameterGrid(param_grid))




### Entrenamiento

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

best_acc = 0.0
best_params = None
best_model = None

for params in ParameterGrid(param_grid):
    print(f"\n🔍 Probando configuración: {params}")
    model = CNN(conv_filters=params['conv_filters'],
                hidden_layers=params['hidden_layers'],
                hidden_units=params['hidden_units']).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=params['lr'])
    criterion = nn.CrossEntropyLoss()

    train_loader = DataLoader(train_data, batch_size=params['batch_size'], shuffle=True)
    val_loader = DataLoader(val_data, batch_size=params['batch_size'], shuffle=False)

    # Entrenamiento simple
    for epoch in range(5):  # ajustable
        model.train()
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}", leave=True)
        for batch_x, batch_y in loop:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            loop.set_postfix(loss=loss.item())

    # Validación
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            outputs = model(batch_x)
            preds = outputs.argmax(dim=1) # obtener la clase con mayor probabilidad
            all_preds.extend(preds.cpu().numpy()) # convertir a numpy y extender la lista
            all_labels.extend(batch_y.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    print(f"✅ Precisión en validación: {acc:.4f}")

    if acc > best_acc:
        best_acc = acc
        best_params = params
        best_model = model

# ===========================
# RESULTADO FINAL
# ===========================
print("\n🏆 Mejor configuración encontrada:")
print(best_params)
print(f"Precisión de validación: {best_acc:.4f}")

# Guardar el mejor modelo
torch.save(best_model.state_dict(), 'best_cnn_mnist.pth')


🔍 Probando configuración: {'batch_size': 64, 'conv_filters': 16, 'hidden_layers': 1, 'hidden_units': 64, 'lr': 0.001}


Epoch 5: 100%|██████████| 750/750 [00:31<00:00, 23.95it/s, loss=0.00384] 


✅ Precisión en validación: 0.9840

🔍 Probando configuración: {'batch_size': 64, 'conv_filters': 16, 'hidden_layers': 1, 'hidden_units': 128, 'lr': 0.001}


Epoch 3:   4%|▍         | 33/750 [00:01<00:30, 23.30it/s, loss=0.0654] 

## Evaluación del modelo

In [ ]:
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test_loader:
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
  print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

Accuracy of the network on the 10000 test images: 99.01%


- https://dudeperf3ct.github.io/cnn/mnist/2018/10/17/Force-of-Convolutional-Neural-Networks/